In [13]:
!pip install --upgrade langchain langchain-ollama langchain-community chromadb
!pip install fastapi nest-asyncio pyngrok uvicorn line-bot-sdk python-dotenv
!pip install fastapi uvicorn python-multipart requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.34
    Uninstalling langchain-core-0.3.34:
      Successfully uninstalled langchain-core-0.3.34
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.18
    Uninstalling langchain-0.3.18:
      Successfully uninstalled langchain-0.3.18
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.3.17
    Uninstalling langchain-community-0.3.17:
      Successfully uninstalled langchain-community-0.3.17


In [2]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

import nest_asyncio
from pyngrok import ngrok
import uvicorn
from fastapi import FastAPI, Request, HTTPException
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage

หั่นคำใส่ LLM

In [3]:
def load_documents(directory):
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory, filename)
            loader = TextLoader(file_path, encoding='utf-8')
            documents.extend(loader.load())
    return documents

docs = load_documents("./")

In [4]:
llm = OllamaLLM(
    model="llama3.2",
    temperature=0.3,
)

embeddings = OllamaEmbeddings(
    model="llama3.2"
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]
)

split_docs = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(split_docs, embeddings)
retriever = vectorstore.as_retriever()

In [5]:
prompt_template = """
คุณคือผู้ให้คำปรึกษา ที่ชื่อว่า KidneyCare ผู้ช่วยให้คำปรึกษาที่มีความรู้ความเข้าใจลึกซึ้งในเรื่องการดูแลสุขภาพของผู้ป่วยโรคไตโดยเน้นไปทางด้านอาหารการกิน การคำนวณสารอาหาร คุณสามารถตอบคำถามเกี่ยวกับโรคไต การดูแลสุขภาพได้ 
โดยคุณจะเชี่ยวชาญและแนะนำได้ดีเป็นพิเศษในเรื่องของอาหารการกินสำหรับผู้ป่วยโรคไต
ตอบคำถามต่อไปนี้โดยยึดตามข้อมูลที่ให้มาเท่านั้น และควรแน่ใจว่าคำตอบถูกต้องตามข้อมูลที่มีอยู่
ถ้าไม่มั่นใจว่ามีข้อมูลก็ควรบอกว่าขออภัยไม่สามารถตอบคำถามนี้ได้ คำถามของคุณจะถูกบันทึกไว้เพื่อปรับปรุงคุณภาพของ AI ของเรา
ใส่อีโมจิต่าง ๆ ในแต่ละคำตอบด้วย เพื่อความเป็นผู้คนมากขึ้น เช่น "✨", "👋"  
และตอบด้วยความชัดเจน กระชับ ในภาษาไทย ลงท้ายด้วยคำว่า "ครับ" เพื่อให้สุภาพ และแทนตัวเองด้วย "ผม" หากคำถามไหนเป็นคำถามง่าย ๆ สามารถตอบได้เลย ไม่ต้องให้คำตอบยาว ๆ แต่ถ้าผู้ใช้ต้องการรายละเอียด ให้ตอบรายละเอียดเพิ่มเติม แต่หากผู้ใช้ไม่บอกว่าต้องการรู้เยอะแค่ไหน ให้ถามก่อนเสมอ


บริบท: {context}

คำถาม: {question}


คำตอบ:
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [6]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

def ask_question(data):
    result = qa_chain({"query": data})
    return result["result"]

In [7]:
question = "ผมเป็นโรคไตเรื้อรังประเภทไหน"

data = f"{question}"
answer = ask_question(data)
print(f"คำถาม: {question}")
print(f"คำตอบ: {answer}")

/var/folders/0h/jv4lpygs6nx8s7rn4yvxrwwm0000gn/T/ipykernel_5334/596302902.py:10: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": data})


คำถาม: ผมเป็นโรคไตเรื้อรังประเภทไหน
คำตอบ: 👋 คุณเป็นโรคไตเรื้อรังระยะ 3a-5 ครับ! นี่หมายความว่าคุณมีระดับฟอสเฟตในเลือดสูงกว่า 4.5 มก./ดล. หรือมากกว่า 5.5 มก./ดล. ในกรณีนี้ คุณควรหลีกเลี่ยงอาหารที่มีส่วนประกอบของ ฟอสฟอรัสชนิดอนินทรีย์ และอาจต้องได้รับยาจับฟอสฟอรัสในทางเดินอาหารเพื่อลดระดับฟอสเฟตในเลือดลงมาอยู่ในระดับปกติครับ


FastAPI เรียกใช้กรณีรูป

In [8]:
from fastapi import FastAPI, File, UploadFile
from dotenv import load_dotenv
import requests
import shutil


load_dotenv()
app = FastAPI()



ROBOFLOW_API_URL = f"https://classify.roboflow.com/foods-qfydk/2?api_key={os.getenv('ROBOFLOW_API_KEY')}"

@app.post("/upload/")
async def upload_image(file: UploadFile = File(...)):

    file_location = f"temp/{file.filename}"
    with open(file_location, "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)


    with open(file_location, "rb") as image_file:
        response = requests.post(ROBOFLOW_API_URL, files={"file": image_file})

    return response.json()


เรียก Line 

In [9]:
from dotenv import load_dotenv
from pyngrok import ngrok
import os
import json

load_dotenv()
app = FastAPI()

line_bot_api = LineBotApi(os.getenv('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(os.getenv('LINE_CHANNEL_SECRET'))
ngrok.set_auth_token(os.getenv('NGROK_AUTH_TOKEN'))

/var/folders/0h/jv4lpygs6nx8s7rn4yvxrwwm0000gn/T/ipykernel_5334/2831993446.py:9: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(os.getenv('LINE_CHANNEL_ACCESS_TOKEN'))
/var/folders/0h/jv4lpygs6nx8s7rn4yvxrwwm0000gn/T/ipykernel_5334/2831993446.py:10: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(os.getenv('LINE_CHANNEL_SECRET'))


In [10]:
import requests
import os
from linebot.models import MessageEvent, ImageMessage
from fastapi import FastAPI
import json

app = FastAPI()

def start_loading_animation(user_id):
    url = "https://api.line.me/v2/bot/chat/loading/start"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {(os.getenv('LINE_CHANNEL_ACCESS_TOKEN'))}"
    }
    data = {"chatId": user_id}
    
    response = requests.post(url, headers=headers, json=data)



def loading_userInformation(user_id):
    rust_server_url = "http://127.0.0.1:7878/chatbot/"
    response = requests.get(f"{rust_server_url}{user_id}")
    global user_information
    user_information = response.text
    return response.json()

def load_roboflow_mapping():
    with open("menuid.json", "r", encoding="utf-8") as file:
        # print("map อยู่")
        return json.load(file)


def get_name(roboflow_id, mapping):
    # print("getnaming")
    return mapping.get(str(roboflow_id)) 


@app.post("/callback")
async def callback(request: Request):
    signature = request.headers['X-Line-Signature']
    body = await request.body()

    try:
        handler.handle(body.decode("utf-8"), signature)
    except InvalidSignatureError:
        raise HTTPException(status_code=400, detail="Invalid signature")

    return 'OK'


# ดึง ROBOFLOW
@handler.add(MessageEvent, message=ImageMessage)
def handle_image_message(event):
    message_id = event.message.id

    user_id = event.source.user_id  
    start_loading_animation(user_id)  

    image_content = line_bot_api.get_message_content(message_id)

    image_path = f"temp_{message_id}.jpg"
    with open(image_path, "wb") as f:
        for chunk in image_content.iter_content():
            f.write(chunk)

    # ส่งไปยัง Roboflow 
    try:
        with open(image_path, "rb") as img:
            response = requests.post(ROBOFLOW_API_URL, files={"file": img})

        result = response.json()

        predict = result.get("predictions", [])
        if predict:
            global roboflow_id
            roboflow_id = predict[0].get("class")
            roboflow_mapping = load_roboflow_mapping()
            name = get_name(roboflow_id, roboflow_mapping)
        else :
            roboflow_id = None

        
    

        if roboflow_id != None : 
            line_bot_api.reply_message (
                event.reply_token,
                TextMessage(text=f"roboflow_id: {roboflow_id} ชื่อไทย: {name}")
            )
        else:
            line_bot_api.reply_message (
                event.reply_token,
                TextMessage(text="ขอโทษด้วยเราไม่สามารถตรวจจับรูปที่ไม่ใช่อาหารได้")
            )

    except Exception as e:
        print(f"err : {e}")

    finally:
        if os.path.exists(image_path):
            os.remove(image_path)  

    




    
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_id = event.source.user_id  
    # print(f"User ID: {user_id}")
    start_loading_animation(user_id)  
    loading_userInformation(user_id)
    

    

    # ดึง rust มาทำงาน
    text = event.message.text


    if text == "สมัครสมาชิก" :
        url = f"https://api.line.me/v2/bot/user/{user_id}/richmenu/richmenu-d14a132432e5dd82bc7b0fed53188eba"
        headers = {
            "Authorization": "Bearer {WzhxAVc0zOkODWYZ66azpLaxU8euQgrB6CR6v/mWkNEiJ7ejx7Njo4lA4GUcKLzZ+65e3bAalkWIwqLBCCzl6EaX+V4AjLEHgAU7vv6jcHYzjIhKV1uqlcr9FK3RA7egLhT1Vi3zkMY/t50HisEdxAdB04t89/1O/w1cDnyilFU=}",
            "Content-Type": "application/json"
        }
        response = requests.post(url, json={}, headers=headers)
    else :
        data = f"{text} {user_information}"
        reply_text = ask_question(data)

        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=reply_text)
        )


ngrok_tunnel = ngrok.connect(8000)
print('Webhook URL:', ngrok_tunnel.public_url + '/callback')

nest_asyncio.apply()

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)










INFO:     Started server process [5334]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Webhook URL: https://e9ac-220-70-170-23.ngrok-free.app/callback
INFO:     147.92.150.196:0 - "POST /callback HTTP/1.1" 200 OK


/var/folders/0h/jv4lpygs6nx8s7rn4yvxrwwm0000gn/T/ipykernel_5334/2108738155.py:135: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(


INFO:     147.92.150.196:0 - "POST /callback HTTP/1.1" 200 OK
INFO:     147.92.150.196:0 - "POST /callback HTTP/1.1" 200 OK
INFO:     147.92.150.196:0 - "POST /callback HTTP/1.1" 200 OK
INFO:     147.92.150.196:0 - "POST /callback HTTP/1.1" 200 OK


t=2025-03-16T20:46:32+0900 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=44d8761ddbb6 err="read tcp 192.168.1.105:57255->3.134.73.173:443: read: connection reset by peer"
t=2025-03-16T20:48:33+0900 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=f25bd2b8ff26 clientid=12322ec4ff469eef9dce02cbe6903f45
t=2025-03-16T20:48:50+0900 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=44d8761ddbb6 err="read EOF from remote peer"
t=2025-03-16T20:48:57+0900 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=454b60b4c006 clientid=12322ec4ff469eef9dce02cbe6903f45
t=2025-03-16T21:10:57+0900 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=44d8761ddbb6 err="read EOF from remote peer"
t=2025-03-16T21:18:58+0900 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=8038f7da04b3 cl